<a href="https://colab.research.google.com/github/penguix0/container-detection/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install dependencies

In [ ]:
!pip install tensorflow
!pip install pillow
!pip install tensorflow-datasets
!pip install numpy
!pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Import libraries

Import all libraries needed and configure the dataset and logging

In [24]:
## Import libaries
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os

# Import the dataset
import container_dataset

# Setup logging
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

bash: cannot set terminal process group (73): Inappropriate ioctl for device
bash: no job control in this shell
/content# ls
'~'   container_dataset   drive   sample_data
/content# cd container_dataset
/content/container_dataset# ls
checksums.tsv         container_dataset_test.py  __pycache__
container_dataset.py  __init__.py
/content/container_dataset# tfds build --register_checksums
2022-11-12 14:48:06.150877: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
INFO[build.py]: Loading dataset  from path: /content/container_dataset/container_dataset.py
2022-11-12 14:48:06.347380: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "NOT_FOUND: Error executing an HTTP request: H

## Import and test the dataset

In [25]:
## Load the dataset
(train_dataset, val_dataset, test_dataset), metadata = tfds.load("container_dataset", 
    split=["train[:80%]", "train[80%:]", "test"], 
    with_info=True)

## Print classes which the program can detect
class_names = metadata.features['objects']["label"].names
print("Class names: {}".format(class_names))

## Print the number of examples in each set
num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples
print("Number of training examples: {}".format(num_train_examples))
print("Number of test examples:     {}".format(num_test_examples))

Class names: ['container_front']
Number of training examples: 85
Number of test examples:     10


## Extract all data from datset

In [26]:
IMG_WIDTH = 100 #pixels
IMG_HEIGHT = 100 #pixels
IMG_DEPTH = 3

train_dataset_img = []
train_dataset_objects = []
## Prepare all images
for i in iter(train_dataset):
    img = i["image"]
    train_dataset_img.append(img)
    objects = []
    for feature in i["objects"]:
        objects.append(i["objects"][feature])
    train_dataset_objects.append(objects)

test_dataset_img = []
test_dataset_objects = []
## Prepare all images
for i in iter(test_dataset):
    img = i["image"]
    test_dataset_img.append(img)
    objects = []
    for feature in i["objects"]:
        objects.append(i["objects"][feature])

    test_dataset_objects.append(objects)

val_dataset_img = []
val_dataset_objects = []
## Prepare all images
for i in iter(val_dataset):
    img = i["image"]
    val_dataset_img.append(img)
    objects = []
    for feature in i["objects"]:
        objects.append(i["objects"][feature])
    val_dataset_objects.append(objects)

train_image_generator      = ImageDataGenerator(rescale=1./255)  # Generator for our training data
test_image_generator       = ImageDataGenerator(rescale=1./255)  # Generator for our test data
validation_image_generator = ImageDataGenerator(rescale=1./255)  # Generator for our validation data



## Create flows from data

In [27]:
BATCH_SIZE = 5 # Number of training examples to process before updating our models variables


train_data_gen = train_image_generator.flow(x=np.asarray(train_dataset_img).astype('float64'),
                                            y=train_dataset_objects,
                                            batch_size=BATCH_SIZE,
                                            shuffle=True)

test_data_gen = validation_image_generator.flow(x=np.asarray(test_dataset_img).astype('float64'),
                                               y=test_dataset_objects,
                                               batch_size=BATCH_SIZE,
                                               shuffle=False)

val_data_gen = validation_image_generator.flow(x=np.asarray(val_dataset_img).astype('float64'),
                                               y=val_dataset_objects,
                                               batch_size=BATCH_SIZE,
                                               shuffle=False)

/usr/local/lib/python3.7/dist-packages/keras/preprocessing/image.py:689: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.y = np.asarray(y)


## Create and compile a model

In [28]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_WIDTH, IMG_HEIGHT, IMG_DEPTH)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(4)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Train the model

In [1]:
EPOCHS = 14

history = model.fit(
    tf.convert_to_tensor(train_data_gen),
    steps_per_epoch=int(np.ceil(len(train_dataset_img) / float(BATCH_SIZE))),
    epochs=EPOCHS,
)

NameError: ignored